# Problem 15: The Forward Pass of a Neural Network

_Version 1.2_

Neural networks are a set of algorithms for pattern recognition. They are loosely inspired by the human brain, and have proven especially useful in data clustering and classification tasks.

In this problem, you will use your knowledge of Python and Numpy to speed-up a common computational kernel that arises in neural networks.

**Example: a "fully connected layer" for images.** Suppose the data we are analyzing consists of $N$ two-dimensional images of size $H \times W$ pixels each. In a neural network, a typical substep is the evaluation of a "fully connected (FC) layer," which takes the images as input and produces a vector of outputs, with one vector per image.

Mathematically, here is a simplified example of what a typical FC layer calculation might look like. Let $x[k, i, j]$ denote the value (e.g., intensity) of the pixel at location $(i, j)$ of the $k$-th input image. Since there are $N$ images, take the values of $k$ to be in the range of 0 to $N-1$, respectively. And since each image is $H \times W$, take $0 \leq i \leq H-1$ and $0 \leq j \leq W-1$. Next, let $\mathrm{out}[k, l]$ denote an output value in the $l$-th element of a vector associated with image $k$, where $0 \leq l \leq M-1$, for some given value of $M$. Lastly, suppose the specific  formula for transforming the input images into this collection of output vectors is given by the formula,

$$
\mathrm{out}[k, l] = b[l] + \sum_{i=0}^{H-1} \sum_{j=0}^{W-1} \left(x[k, i, j] \times w[l, i, j]\right)
$$

where $w[l, i, j]$ are "weights" and $b[l]$ are "biases." The process of "training" the neural network from sample data determines these weight and bias parameters, but for this problem, just assume that they are given.

If it's helpful, here is a picture of what this formula is doing for each $(k, l)$ pair:
<img src = "fully_connected.png" width = "600">

**The baseline implementation.** In the code cells below, we define a Python function, `FC_naive(x, w, b)`, that implements the FC layer calculation from above using a straightforward, albeit somewhat naive, method. Your goal is to make this baseline run faster.

To start, first run the next three code cells to estimate the time of the baseline implementation.

In [5]:
from itertools import count
for i in count(5):
    if i > 10: break
    print(i, end = ' ')

5 6 7 8 9 10 

In [6]:
import numpy as np
import time

def rel_error(x, y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [7]:
def FC_naive(x, w, b):
    """
    Inputs:
    - x: A numpy array of images of shape (N, H, W)
    - w: A numpy array of weights of shape (M, H, W)
    - b: A numpy vector of biases of size M

    Returns: 
    - out: a numpy array of shape (N, M)
    """
    N, H, W = x.shape
    M, _, _ = w.shape
    out = np.zeros((N,M))
    for ni in range(N):
        for mi in range(M):
                out[ni,mi] = b[mi]
                for d1 in range(H):
                    for d2 in range(W):
                        out[ni,mi] += x[ni, d1, d2] * w[mi, d1, d2] 
    return out

In [8]:
num_inputs = 50
input_shape = (128, 256)
output_dim = 10

x = np.random.rand(num_inputs, *input_shape)
w = np.random.rand(output_dim, *input_shape)
b = np.random.rand(output_dim)
start_time = time.time ()
out = FC_naive(x, w, b)
elapsed_time = time.time () - start_time
print ("==> Took %g seconds." % elapsed_time)

==> Took 12.7335 seconds.


**Exercise 1** (5 points). Let's start by seeing if we can make `FC_naive()` function faster by rewriting the two innermost loops, i.e., the `d1` and `d2` loops:

```python
for d1 in range(H):
    for d2 in range(W):
        out[ni, mi] += x[ni, d1, d2] * w[mi, d1, d2]
```

For this exercise, complete the function `two_inner_loops(x_i, w_l, b_j)`, below, so that it implements the same computation as these two `d1` and `d2` loops, but is much faster. It should return `out[ni, mi]`. The input `x_i` is the `i`-th image, `w_l` is the `l`-th weight matrix, and `b_l` is the `l`-th component of the bias vector.

The test code will check your results and benchmark a complete FC layer using the function `FC_two_loops()`, defined below. You'll see that it calls your `two_inner_loops()` routine to implement the two innermost loops.

To get credit on this exercise, the resulting execution time of `FC_two_loops()` must be at least **100 times faster** than `FC_naive()` on the problem sizes being tested below when running on the Vocareum platform. There is no partial credit for smaller speedups. Having said that, a little bit of basic Numpy should go a long way.

In [9]:
def two_inner_loops(x_i, w_l, b_l):
    """
    Inputs:
    - x_i: A numpy array of images of shape (H, W)
    - w_l: A numpy array of weights of shape (H, W)
    - b_l: A float (single number)

    Returns: 
    - out: A float (single number)
    """
    return np.sum(np.multiply(x_i, w_l)) + b_l


In [10]:
# Test cell: 'FC_two_loops_1' (5 points)

def FC_two_loops(x, w, b):
    """
    Inputs:
    - x: A numpy array of images of shape (N, H, W)
    - w: A numpy array of weights of shape (M, H, W)
    - b: A numpy vector of biases of size M

    Returns: 
    - out: a numpy array of shape (N, M)
    """
    N, H, W = x.shape
    M, _, _ = w.shape
    out = np.zeros((N,M))
    for ni in range(N):
           for mi in range(M):
                out[ni, mi] = two_inner_loops(x[ni,  :, :], w[mi,  :, :], b[mi])
    return out

num_inputs = 50
input_shape = (128, 256)
output_dim = 10

x = np.random.rand(num_inputs, *input_shape)
w = np.random.rand(output_dim, *input_shape)
b = np.random.rand(output_dim)

print("Checking the correctness of your implementation...")
out_fast = FC_two_loops(x, w, b)
out_naive = FC_naive(x, w, b)
error = rel_error(out_naive, out_fast)
print("==> Output error:", error)
assert error < 1e-12, "The value of your output is incorrect or not accurate enough"
print("==> This level of error is acceptable.")

print("\nBenchmarking your code...")
T_fast = %timeit -o FC_two_loops(x, w, b)
#start_time = time.time ()
#for i in range(5):
#    out_fast = FC_two_loops(x, w, b)
#elapsed_time_fast = (time.time () - start_time)/5
elapsed_time_fast = np.average(T_fast.all_runs) / T_fast.loops
print ("==> Took %g seconds." % elapsed_time_fast)

print("\nBenchmarking the naive code...")
T_naive = %timeit -o FC_naive(x, w, b)
#start_time = time.time ()
#for i in range(5):
#    out_naive = FC_naive(x, w, b)
#elapsed_time_naive = (time.time () - start_time)/5
elapsed_time_naive = np.average(T_naive.all_runs) / T_naive.loops
print ("==> Took %g seconds." % elapsed_time_naive)

speed_up = elapsed_time_naive/elapsed_time_fast
print("Speed-up:", speed_up)
assert speed_up >= 100, "The speed-up of your method is less than 100"

print("\n(Passed!)")

Checking the correctness of your implementation...
==> Output error: 6.344439372877484e-15
==> This level of error is acceptable.

Benchmarking your code...
18.5 ms ± 521 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
==> Took 0.0185347 seconds.

Benchmarking the naive code...
12.2 s ± 194 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
==> Took 12.2305 seconds.
Speed-up: 659.8674341373804

(Passed!)


**Question 2** (5 points). Now, completely rewrite the `FC_naive()` function by at least **2,000 times**.

> This improvement can be attained with basic Numpy operations that you've learned (i.e., no "new" functions) and no explicit loops.

In [101]:
def FC_no_loop(x, w, b):
    """
    Inputs:
    - x: A numpy array of images of shape (N, H, W)
    - w: A numpy array of weights of shape (M, H, W)
    - b: A numpy vector of biases of size M

    Returns: 
    - out: a numpy array of shape (N, M)
    """
    N, H, W = x.shape
    M, _, _ = w.shape
    out = np.zeros((N,M))
    x = np.reshape(x, (N, H*W))
    w = np.reshape(w, (M, H*W))
    #out = x @ w.T + b
    out = np.matmul(x, w.T) + b
    return out

In [99]:
num_inputs = 4
output_dim = 2
input_shape = (3, 5)
x = np.random.rand(num_inputs, *input_shape)
w = np.random.rand(output_dim, *input_shape)
b = np.random.rand(output_dim)

N, H, W = x.shape
M, _, _ = w.shape
(N, H*W)
#x.shape
#out = np.zeros((N,M))
#x = np.reshape(x, (N, H*W))
#w = np.reshape(w, (M, H*W))
#out = x @ w.T + b
##out
np.reshape(x, (N, H*W))

#return (np.sum((X[:, np.newaxis, :] - X[np.newaxis, :, :]) ** 2, axis = -1))**.5

array([[0.03914401, 0.5864001 , 0.71874931, 0.11727744, 0.82320592,
        0.2437209 , 0.62509886, 0.76155648, 0.05291754, 0.49064931,
        0.78501498, 0.94810241, 0.73740535, 0.19162696, 0.90076777],
       [0.31776557, 0.86562932, 0.38322886, 0.78061744, 0.11184559,
        0.64858118, 0.84699732, 0.91938707, 0.91389242, 0.45003696,
        0.08959259, 0.2265762 , 0.58623963, 0.36740506, 0.77039268],
       [0.40074013, 0.30371424, 0.69846621, 0.54899445, 0.23226717,
        0.65208801, 0.66517656, 0.20873663, 0.86778472, 0.46221803,
        0.96941706, 0.9453924 , 0.22766548, 0.60950564, 0.06304842],
       [0.9593062 , 0.51081238, 0.62722386, 0.1265792 , 0.32704114,
        0.09925006, 0.56438055, 0.20119598, 0.36590146, 0.76860622,
        0.05584117, 0.37972464, 0.98830831, 0.42552853, 0.99561701]])

In [89]:
x

array([[[0.44726217, 0.47200521, 0.68538702, 0.2878556 , 0.11058171],
        [0.36614354, 0.75822343, 0.51937557, 0.27832948, 0.29582117],
        [0.02182309, 0.31758685, 0.0199058 , 0.58921998, 0.26170271]],

       [[0.96191295, 0.69288992, 0.24357373, 0.00643333, 0.83717509],
        [0.13992871, 0.52638467, 0.92951979, 0.83248554, 0.82927749],
        [0.648922  , 0.35620569, 0.75585358, 0.19428066, 0.36018549]],

       [[0.36457812, 0.20437979, 0.70242075, 0.7912644 , 0.65078227],
        [0.52069965, 0.12443479, 0.1358343 , 0.38920819, 0.19624455],
        [0.48782709, 0.4691875 , 0.01997237, 0.34066853, 0.92549019]],

       [[0.59774719, 0.37651439, 0.53459833, 0.08198689, 0.8335332 ],
        [0.49476061, 0.99848124, 0.10040248, 0.73720644, 0.78363202],
        [0.05020277, 0.47191501, 0.39441538, 0.78070692, 0.5835159 ]]])

In [90]:
x[:]

array([[[0.44726217, 0.47200521, 0.68538702, 0.2878556 , 0.11058171],
        [0.36614354, 0.75822343, 0.51937557, 0.27832948, 0.29582117],
        [0.02182309, 0.31758685, 0.0199058 , 0.58921998, 0.26170271]],

       [[0.96191295, 0.69288992, 0.24357373, 0.00643333, 0.83717509],
        [0.13992871, 0.52638467, 0.92951979, 0.83248554, 0.82927749],
        [0.648922  , 0.35620569, 0.75585358, 0.19428066, 0.36018549]],

       [[0.36457812, 0.20437979, 0.70242075, 0.7912644 , 0.65078227],
        [0.52069965, 0.12443479, 0.1358343 , 0.38920819, 0.19624455],
        [0.48782709, 0.4691875 , 0.01997237, 0.34066853, 0.92549019]],

       [[0.59774719, 0.37651439, 0.53459833, 0.08198689, 0.8335332 ],
        [0.49476061, 0.99848124, 0.10040248, 0.73720644, 0.78363202],
        [0.05020277, 0.47191501, 0.39441538, 0.78070692, 0.5835159 ]]])

In [97]:
(np.reshape(x, (N, -1)) == np.reshape(x, (N, H*W))).all()

True

In [102]:
# Test cell: 'FC_no_loop' (5 points)
num_inputs = 50
input_shape = (128, 256)
output_dim = 10

x = np.random.rand(num_inputs, *input_shape)
w = np.random.rand(output_dim, *input_shape)
b = np.random.rand(output_dim)

print("Checking the correctness of your implementation...")
out_fast = FC_no_loop(x, w, b)
out_naive = FC_naive(x, w, b)
error = rel_error(out_naive, out_fast)
print("==> Output error:", error)
assert error < 1e-12, "The value of your output is incorrect or not accurate enough"
print("==> This level of error is acceptable.")

print("\nBenchmarking your code...")
T_fast = %timeit -o FC_no_loop(x, w, b)
elapsed_time_fast = np.average(T_fast.all_runs) / T_fast.loops
print ("==> Took %g seconds." % elapsed_time_fast)

print("\nBenchmarking the naive code...")
T_naive = %timeit -o FC_naive(x, w, b)
elapsed_time_naive = np.average(T_naive.all_runs) / T_naive.loops
print ("==> Took %g seconds." % elapsed_time_naive)

speed_up = elapsed_time_naive/elapsed_time_fast
print("Speed-up:", speed_up)
assert speed_up >= 2000, "The speed-up of your method is less than 2000"

print("\n(Passed!)")

Checking the correctness of your implementation...
==> Output error: 6.833786658688915e-15
==> This level of error is acceptable.

Benchmarking your code...
1.6 ms ± 31.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
==> Took 0.00160223 seconds.

Benchmarking the naive code...
14.7 s ± 612 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
==> Took 14.6761 seconds.
Speed-up: 9159.833853490416

(Passed!)


**Fin!** You've reached the end of this problem. Don't forget to restart the
kernel and run the entire notebook from top-to-bottom to make sure you did
everything correctly. If that is working, try submitting this problem. (Recall
that you *must* submit and pass the autograder to get credit for your work!)